In [1]:
import numpy as np 
import pandas as pd 
import math
import uncertainties
from uncertainties import ufloat_fromstr
from uncertainties import unumpy as unp
from uncertainties import ufloat
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from scipy import stats
from scipy . optimize import curve_fit
from scipy . interpolate import interp1d
from scipy import optimize

# --- plot parameters --- 
plt.rcParams["figure.figsize"] = (15,10)
plt.rc('font', family = 'serif', serif = 'cmr10') 
plt.rcParams.update({'font.size': 22})
plt.rcParams['axes.unicode_minus'] = False

In [2]:
# --- Importing Lab Data --- 

Green_Laser = np.loadtxt("Data\Green_Laser.dat", unpack = True)
HeNe_Laser = np.loadtxt("Data\HeNe_Laser.dat", unpack = True)
Blue_Laser = np.loadtxt("Data\Blue_Laser.txt", unpack = True)

# --- Time interval --- 

t = np.linspace(0, 10, 1000)

# --- Supplementary Data --- 

HeNe_Motor = ufloat(3.567e-4, 1.3e-6)
Green_Motor = ufloat(3.670e-4, 1.3e-6)
Blue_Motor = ufloat(2.450e-4, 1.3e-6)
#--- 
HeNe_Motor_time = ufloat(621.5, 2)
Green_Motor_time = ufloat(775, 2)
Blue_Motor_time = ufloat(1523, 2)

In [3]:
def PathDifference(data, time_motor, time_sample, sample_rate, distance): # Where distance is the start and end point of mirror over a time interval. 
    xt = distance/time_motor #returns the mirror speed in m/s for each travel. 
    dt = time_sample/sample_rate
    return xt*dt*1e9

In [4]:
'{:.3}'.format(PathDifference(Green_Motor, Green_Motor_time, 10, 1000, Green_Motor))

'4.74+/-0.02'

In [5]:
'{:.3}'.format(PathDifference(Blue_Motor, Blue_Motor_time, 10, 1000, Blue_Motor))

'1.61+/-0.01'

In [6]:
def freq(d_t, t_t, t_s, N):
     return (2*(d_t/t_t)*t_s)/N #Where, d_t is dist. travelled by motor, t_t travel duration, t_s sampling duration, N sample rate. 

def fft(data):
     ft_data = np.fft.fft(data)
     ft_amp = np.abs(ft_data)
     ft_freq = np.abs(np.fft.fftfreq(int(len(ft_data))))
     ft_waves = 1/ft_freq 
     return ft_data, ft_amp, ft_freq, ft_waves

def wavelength(dist, t, tsamp, N, ft_amp, a, b): 
    dx = (((2*dist)/t)*tsamp)/N

    λ = []
    Δλ = []

    for i in range(0, N):
        λv = N*(dx/(i + 1))
        Δλv = N*(dx/((i + 1)*(i + 2)))
        λ.append(λv)
        Δλ.append(Δλv)

    red = np.array(λ)*1e9 # --- scaling to nm for better illustration 
    Δλ = np.array(Δλ)*1e9

    na = []
    nw = []
    
    for i in range(0, len(red)):
        if a > red[i] > b:
            na.append(ft_amp[i])
            nw.append(red[i])

    ft_data_f = [na, nw]
    corresponding_λ = nw[np.argmax(ft_data_f[0])]
    print('corresponding λ is', '{:.3f}'.format(corresponding_λ), 'nm')

    x = (nw[np.argmax(ft_data_f[0])]/2)*(nw[np.argmax(ft_data_f[0])] - nw[np.argmax(ft_data_f[0]) + 1])

    return na, nw, corresponding_λ, Δλ, x 

In [7]:
ft_HeNe = fft(HeNe_Laser)
HeNe_Wavelength = wavelength(HeNe_Motor, HeNe_Motor_time, 10, 1000, ft_HeNe[1], 2000, 50)

ft_Green = fft(Green_Laser)
Green_Wavelength = wavelength(Green_Motor, Green_Motor_time, 10, 1000, ft_Green[1], 5000, 50)

ft_Blue = fft(Blue_Laser)
Blue_Wavelength = wavelength(Blue_Motor, Blue_Motor_time, 10, 1000, ft_Blue[1], 3000, 50)

corresponding λ is 604.141+/-2.937 nm
corresponding λ is 473.548+/-2.075 nm
corresponding λ is 402.167+/-2.198 nm


# Calculating other factors. 

## $$(\Delta \lambda)_{min} = n_t \lambda^2 / x_t (N - 1) $$ 

## $$ \lambda_{min} \ = \ 2(x_t/n_t) $$

In [8]:
def min_res_wavelength(wavelength, n, x, N):
    return (n*wavelength**2)/(x*(N - 1))

def xt(distance, time_motor):
    return distance/time_motor*1e9

In [9]:
xt_HeNe = xt(HeNe_Motor, HeNe_Motor_time)
print('λ_min HeNe:', (xt_HeNe/100)*2, 'nm')
MRW_HeNe = min_res_wavelength(400, 100, xt_HeNe, 1000)
print('HeNe MRW:', MRW_HeNe, 'nm')

xt_Green = xt(Green_Motor, Green_Motor_time)
print('λ_min Green:', (xt_Green/100)*2, 'nm')
MRW_Green = min_res_wavelength(400, 100, xt_Green, 1000)
print('Green MRW:', MRW_Green, 'nm')

xt_Blue = xt(Blue_Motor, Blue_Motor_time)
print('λ_min Blue:', (xt_Blue/100)*2, 'nm')
MRW_Blue = min_res_wavelength(400, 100, xt_Blue, 1000)
print('Blue MRW:', MRW_Blue, 'nm')


λ_min HeNe: 11.48+/-0.06 nm
HeNe MRW: 27.91+/-0.14 nm
λ_min Green: 9.47+/-0.04 nm
Green MRW: 33.82+/-0.15 nm
λ_min Blue: 3.217+/-0.018 nm
Blue MRW: 99.6+/-0.5 nm
